In [ ]:
%load_ext autoreload
%autoreload 2    

In [ ]:
import os, time, shutil, sys
from pathlib import Path
from glob import glob
from pprint import pprint
import numpy as np
import spikeinterface.full as si
import matplotlib.pyplot as plt
import h5py

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import spike_sorting as ss
import axon_tracking.template_extraction as te

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/concatenated/'
for w in range(24):
    try:
        stream_id = 'well' + str(w).zfill(3)
        te.extract_templates_from_concatenated_recording(root_path, stream_id)
        print(stream_id)
    except Exception as e:
        print(e)
        continue

In [ ]:
sorter = 'kilosort2_5'
si.Kilosort2_5Sorter.set_kilosort2_5_path('/home/phornauer/Git/Kilosort_2020b') #Change
sorter_params = si.get_default_sorter_params(si.Kilosort2_5Sorter)
sorter_params['n_jobs'] = -1
sorter_params['detect_threshold'] = 6
sorter_params['minFR'] = 0.01
sorter_params['minfr_goodchannels'] = 0.01
sorter_params['keep_good_only'] = False
sorter_params['do_correction'] = False
sorter_params['NT'] = 64*2*1024 + 64
#pprint(sorter_params)

In [ ]:
axon_path = '/net/bs-filesvr02/export/group/hierlemann/recordings/Maxtwo/achernov/Tabea/DCZ_Row2/231214/T002443/AxonTracking/000068/data.raw.h5'
nw_sorting_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/'
stream_ids = ss.get_stream_ids(axon_path)

save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/concatenated/'

In [ ]:
for stream_id in stream_ids:
    well_recording_path = os.path.join(nw_sorting_path, stream_id, 'spikeinterface_recording.json')
    if os.path.exists(well_recording_path):
        concat_rec = si.load_extractor(well_recording_path, base_folder=True)
        axon_rec, ax_el, pos = ss.concatenate_recording_slices(axon_path, stream_id)
        sliced = ss.cut_concatenated_recording(concat_rec, cutout=600)
        concatenated = ss.intersect_and_concatenate_recording_list([axon_rec, sliced])
        
        ss.clean_sorting(rec=concatenated, save_root=save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)

In [ ]:
si.run_sorter(

In [ ]:
axon_rec, ax_el, pos = ss.concatenate_recording_slices(rec_path, stream_id)

In [ ]:
sliced = ss.cut_concatenated_recording(concat_rec, cutout=600)

In [ ]:
concatenated = ss.intersect_and_concatenate_recording_list([axon_rec, sliced])
concatenated

In [ ]:
concatenation_list = []
for i, path in enumerate(path_list[:-1]):
    h5 = h5py.File(path)
    rec_name = list(h5['wells'][stream_id].keys())[0]
    rec = si.MaxwellRecordingExtractor(path, stream_id=stream_id, rec_name=rec_name)
    if i > 0:
        rec = rec.frame_slice(start_frame = rec.get_num_frames() - 10*60*rec.get_sampling_frequency(), end_frame = rec.get_num_frames())
    #rec_binary = rec.save(format="memory",n_jobs=-1)
    rec_binary = rec.save(format="binary",folder='/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/'+ str(i+1), n_jobs=-1)
    print('Saved binary')
    slice_rec_binary = rec_binary.channel_slice(nw_channels, renamed_channel_ids=list(range(len(nw_channels))))
    print('Sliced binary')
    slice_centered = si.center(slice_rec_binary)
    print('Centered binary')
    concatenation_list.append(slice_centered)

In [ ]:
concatenation_list.append(axon_rec)

In [ ]:
full_concat = si.concatenate_recordings(concatenation_list)

In [ ]:
save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/ax-bl_test'
ss.clean_sorting(rec=conc, save_root=save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)

In [ ]:
recording_concat_loaded = si.load_extractor('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/ax-bl_test/well002/spikeinterface_recording.json')

In [ ]:
sorting_concat_loaded = si.KiloSortSortingExtractor('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/ax-bl_test/well002/sorter_output')

In [ ]:
cleaned_sorting = si.remove_excess_spikes(sorting_concat_loaded, recording_concat_loaded)
cleaned_sorting.register_recording(recording_concat_loaded)

In [ ]:
segment_sorting = si.SplitSegmentSorting(cleaned_sorting, recording_concat_loaded)

In [ ]:
nw_sort = si.SelectSegmentSorting(segment_sorting, 1)

In [ ]:
nw_sort = si.remove_excess_spikes(nw_sort, recording_concat_loaded.recording_list[1])

In [ ]:
nw_sort.register_recording(recording_concat_loaded.recording_list[1])

In [ ]:
recording_concat_loaded.recording_list[0]._parent_recording.recording_list[0]

In [ ]:
axon_rec, ax_el = ss.concatenate_recording_slices(rec_path, stream_id)

In [ ]:
network_rec = ss.concatenate_recording_list(path_list, stream_id)

In [ ]:
nw_rec_el = network_rec.get_property("contact_vector")["electrode"]
ax_rec_el = axon_rec.get_property("contact_vector")["electrode"]

common_el = list(set(ax_rec_el).intersection(nw_rec_el))

chan_idx = [np.where(ax_rec_el == el)[0][0] for el in common_el]
ax_channels = axon_rec.get_channel_ids()[chan_idx]
ax_rec_slice = axon_rec.channel_slice(ax_channels, renamed_channel_ids=list(range(len(chan_idx))))

rec_list = [ax_rec_slice.astype('float16')]

chan_idx = [np.where(nw_rec_el == el)[0][0] for el in common_el]
nw_channels = network_rec.get_channel_ids()[chan_idx]
nw_rec_slice = network_rec.channel_slice(nw_channels, renamed_channel_ids=list(range(len(chan_idx))))

rec_list.append(nw_rec_slice.astype('float16'))

In [ ]:
concatenated_recording = si.concatenate_recordings(rec_list)

In [ ]:
save_root = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test1'
ss.clean_sorting(rec=concatenated_recording, save_root=save_root, stream_id=stream_id, sorter=sorter, sorter_params=sorter_params)

In [ ]:
#'/net/bs-filesvr02/export/group/hierlemann/recordings/Mea1k/phornauer/Chemogenetics/231205/2977/Cell_B_positive.raw.h5'
rec_path, stream_id, rec_name = path_list[0],'well000','rec0000'
rec = si.MaxwellRecordingExtractor(rec_path, stream_id=stream_id, rec_name=rec_name)

In [ ]:
rec

In [ ]:
save_path = Path('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test')
rec.save(folder=save_path, n_jobs=32, chunk_size=20000)

In [ ]:
frame_sliced = rec.frame_slice(start_frame=0*rec.get_sampling_frequency(),end_frame=rec.get_num_samples())
save_path = Path('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test3')
frame_sliced.save(folder=save_path, n_jobs=32, chunk_size=20000)

In [ ]:
rec

In [ ]:
save_path = Path('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test')
loaded_frame = si.load_extractor(save_path)
loaded_frame

In [ ]:
sel_channels = np.random.choice(rec.get_channel_ids(),size=355,replace=False)
slice_rec = rec.channel_slice(sel_channels, renamed_channel_ids=list(range(len(sel_channels))))
slice_rec

In [ ]:
slice_save_path = Path('/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_2/concatenated/test1')
slice_rec.save_to_folder(folder=slice_save_path, n_jobs=-1)

In [ ]:
rec_binary = rec.save(format="memory",n_jobs=-1)

In [ ]:
sel_channels = np.random.choice(rec.get_channel_ids(),size=355,replace=False)
slice_rec_binary = rec_binary.channel_slice(sel_channels, renamed_channel_ids=list(range(len(sel_channels))))
slice_rec_binary